**2025 Military AI CONtest**<br>
AI Autonomous Driving – Pre-Training Code Sample<br> 
NVIDIA Jetson Nano/Wingbot/JupyterLab<br>
Version : v1.0<br>
File : robot_camera_qrcode.ipynb


In [1]:
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from collections import deque

pipeline = (
    "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, format=NV12, framerate=30/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
)

cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)

if not cap.isOpened():
    print("Camera Open Error")
    exit()

video_widget = widgets.Image(format='jpeg', layout=widgets.Layout(width='640px', height='480px'))
display(video_widget)

def frame_to_bytes(frame):
    _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
    return buf.tobytes()

qr_detector = cv2.QRCodeDetector()
log_buffer = deque(maxlen=5)

frame_count = 0 

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        frame = cv2.flip(frame, -1)

        data, bbox = None, None
        if frame_count % 3 == 0:
            data, bbox, _ = qr_detector.detectAndDecode(frame)

        if data:
            current_time = time.strftime("%H:%M:%S")
            log_buffer.append(f"[{current_time}] Detected QR Code: {data}")

            if bbox is not None:
                pts = bbox.astype(int).reshape(-1, 2)
                for i in range(len(pts)):
                    cv2.line(frame, tuple(pts[i]), tuple(pts[(i + 1) % len(pts)]), (0, 255, 0), 2)
                cv2.putText(frame, data, (pts[0][0], pts[0][1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            clear_output(wait=True)
            display(video_widget)
            for line in log_buffer:
                print(line)
        video_widget.value = frame_to_bytes(frame)

except KeyboardInterrupt:
    print("Camera stream exit")

finally:
    cap.release()


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x14\x0e\x0…

[19:07:10] Detected QR Code: testqr
[19:07:10] Detected QR Code: testqr
[19:07:10] Detected QR Code: testqr
[19:07:10] Detected QR Code: testqr
[19:07:10] Detected QR Code: testqr
Camera stream exit
